<a href="https://colab.research.google.com/github/JidaphaMekon/Oppa-Workshop/blob/main/Linear_interp_lag7days.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas_market_calendars
!pip install yfinance scikit-learn xgboost matplotlib
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pandas_market_calendars as mcal
import matplotlib.pyplot as plt

# ไลบรารีสำหรับ Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.9/208.9 kB 9.9 MB/s eta 0:00:00


In [ ]:

# ------------------------------
# 0. กำหนด list หุ้นทั้งหมด
# ------------------------------
tickers = ["AAPL", "AMD", "AVGO", "GOOG", "GOOGL", "META", "MSFT", "NVDA"]

# กำหนดช่วงวันที่
start_date = datetime(2019, 1, 1)
end_date   = datetime(2024, 12, 31)

all_data_list = []

# ------------------------------
# 1. ดาวน์โหลดข้อมูลหุ้นทั้งหมด
# ------------------------------
for ticker in tickers:
    print(f"📥 Downloading {ticker} ...")
    data = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))

    if data.empty:
        print(f"❌ No data for {ticker}")
        continue

    data.reset_index(inplace=True)

    if isinstance(data.columns, pd.MultiIndex):
        data.columns = [' '.join(col).strip() if isinstance(col, tuple) else col for col in data.columns.values]

    def clean_columns(cols):
        cleaned = []
        for col in cols:
            if 'Date' in col:
                cleaned.append('Date')
            else:
                cleaned.append(col.split()[0])
        return cleaned

    data.columns = clean_columns(data.columns)
    data['Symbol'] = ticker.upper()

    wanted_cols = ['Date', 'Open','Close','High', 'Low', 'Symbol']
    data = data[[col for col in wanted_cols if col in data.columns]]

    all_data_list.append(data)

# Concatenate all dataframes in the list into a single dataframe
flat_df = pd.concat(all_data_list, ignore_index=True)



/tmp/ipython-input-1844275127.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1844275127.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1844275127.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1844275127.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tic

📥 Downloading AAPL ...
📥 Downloading AMD ...
📥 Downloading AVGO ...
📥 Downloading GOOG ...



/tmp/ipython-input-1844275127.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1844275127.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1844275127.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1844275127.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ti

📥 Downloading GOOGL ...
📥 Downloading META ...
📥 Downloading MSFT ...
📥 Downloading NVDA ...


[*********************100%***********************]  1 of 1 completed


In [ ]:
import pandas as pd
import pandas_market_calendars as mcal

# 1. สร้าง continuous date range
all_dates = pd.DataFrame({'Date': pd.date_range(flat_df['Date'].min(), flat_df['Date'].max())})

# 2. ขยาย Symbol list ให้ครบทุกวัน
symbols = flat_df['Symbol'].unique()
expanded_list = []
for sym in symbols:
    temp = all_dates.copy()
    temp['Symbol'] = sym
    expanded_list.append(temp)
all_dates_symbols = pd.concat(expanded_list, ignore_index=True)

# 3. merge กับ flat_df ของราคาหุ้น
full_df = all_dates_symbols.merge(flat_df, on=['Date','Symbol'], how='left')
full_df = full_df[['Date', 'Open','Close','High', 'Low', 'Symbol']]

In [ ]:
full_df

,Date,Open,Close,High,Low,Symbol
0,2019-01-02,36.854254,37.575207,37.796491,36.697214,AAPL
1,2019-01-03,34.258351,33.832443,34.672365,33.787234,AAPL
2,2019-01-04,34.389213,35.276722,35.345726,34.215519,AAPL
3,2019-01-05,NaN,NaN,NaN,NaN,AAPL
4,2019-01-06,NaN,NaN,NaN,NaN,AAPL
...,...,...,...,...,...,...
17515,2024-12-26,139.669575,139.899521,140.819334,137.700003,NVDA
17516,2024-12-27,138.519837,136.980164,138.989736,134.680677,NVDA
17517,2024-12-28,NaN,NaN,NaN,NaN,NVDA
17518,2024-12-29,NaN,NaN,NaN,NaN,NVDA


In [ ]:
# ----------------------------
# เติมค่าว่าง Close แยกแต่ละ Symbol
# ----------------------------
price_cols = ['Open','Close','High', 'Low']
for col in price_cols:
    full_df[col] = full_df.groupby('Symbol')[col].transform(
        lambda x: x.interpolate(method='linear')  # เติมค่า linear ระหว่างข้อมูล
    )

In [ ]:
full_df.to_excel("full_df.xlsx", index=False)
print("✅ Exported full_df.xlsx")

✅ Exported full_df.xlsx


In [ ]:
full_df

,Date,Open,Close,High,Low,Symbol
0,2019-01-02,36.854254,37.575207,37.796491,36.697214,AAPL
1,2019-01-03,34.258351,33.832443,34.672365,33.787234,AAPL
2,2019-01-04,34.389213,35.276722,35.345726,34.215519,AAPL
3,2019-01-05,34.719948,35.250549,35.367934,34.382076,AAPL
4,2019-01-06,35.050683,35.224377,35.390142,34.548633,AAPL
...,...,...,...,...,...,...
17515,2024-12-26,139.669575,139.899521,140.819334,137.700003,NVDA
17516,2024-12-27,138.519837,136.980164,138.989736,134.680677,NVDA
17517,2024-12-28,137.280101,137.140127,139.406306,134.450720,NVDA
17518,2024-12-29,136.040364,137.300090,139.822876,134.220764,NVDA


In [ ]:
flat_df=full_df

In [ ]:
# สร้าง lag ของราคาย้อนหลัง 7 วัน (Close_lag1 ... Close_lag7)
for lag in range(1, 8):  # 1 ถึง 7 วัน
    flat_df[f'Close_lag{lag}'] = flat_df.groupby('Symbol')['Close'].shift(lag)
# 9. สร้าง Target: Close ของวันถัดไป
flat_df['Close_next'] = flat_df.groupby('Symbol')['Close'].shift(-1)

# 10. สร้าง Feature Matrix X
lag_cols = [f'Close_lag{i}' for i in range(1,8)]  # Close_lag1 ... Close_lag7

X_cols = lag_cols
X = flat_df[X_cols]
# Target
y = flat_df['Close_next']

# ตรวจสอบตัวอย่าง
print("Feature Matrix X:")
print(X.head(10))
print("\nTarget y:")
print(y.head(10))

# 11. ลบ row ที่มี NaN ใน X หรือ y
data_ml = pd.concat([X, y], axis=1)  # รวม X และ y ชั่วคราว
data_ml_clean = data_ml.dropna().reset_index(drop=True)

# แยกกลับเป็น X และ y
X_clean = data_ml_clean[X_cols]
y_clean = data_ml_clean['Close_next']

# ตรวจสอบตัวอย่าง
print("Feature Matrix X (clean):")
print(X_clean.head(10))
print("\nTarget y (clean):")
print(y_clean.head(10))

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# สำหรับ LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# สำหรับ Prophet
from prophet import Prophet


# ===============================
# ⚙️ Sliding Window Parameters
# ===============================
window_train = 0.7
window_test = 0.15

results = []

# ===============================
# 🤖 ML Models
# ===============================
ml_models = {
    'Linear': LinearRegression(),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42, verbosity=0),
    'SVR': SVR()
}

# ===============================
# 🔁 Loop by Symbol
# ===============================
for sym in flat_df['Symbol'].unique():
    df_sym = flat_df[flat_df['Symbol']==sym].sort_values('Date').reset_index(drop=True)
    df_sym = df_sym.dropna(subset=X_cols + ['Close_next']).reset_index(drop=True)

    X_clean = df_sym[X_cols]
    y_clean = df_sym['Close_next']

    n = len(X_clean)
    step = int(n * window_test)
    start = 0

    while start + int(n * window_train) + step <= n:
        train_idx = range(start, start + int(n * window_train))
        test_idx  = range(start + int(n * window_train), start + int(n * window_train) + step)

        X_train = X_clean.iloc[train_idx].values
        y_train = y_clean.iloc[train_idx].values
        X_test  = X_clean.iloc[test_idx].values
        y_test  = y_clean.iloc[test_idx].values

        # Scale เฉพาะ train
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled  = scaler.transform(X_test)

        # ===============================
        # 🧠 1. ML Models
        # ===============================
        for name, model in ml_models.items():
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
            results.append({
                'Symbol': sym,
                'model': name,
                'start_idx': start,
                'rmse': np.sqrt(mean_squared_error(y_test, y_pred)),
                'mae': mean_absolute_error(y_test, y_pred),
                'r2': r2_score(y_test, y_pred)
            })

        # ===============================
        # 🔮 2. LSTM
        # ===============================
        X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
        X_test_lstm  = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

        lstm_model = Sequential([
            LSTM(64, activation='relu', input_shape=(1, X_train_scaled.shape[1])),
            Dense(1)
        ])
        lstm_model.compile(optimizer='adam', loss='mse')
        lstm_model.fit(X_train_lstm, y_train, epochs=20, batch_size=16, verbose=0)

        y_pred_lstm = lstm_model.predict(X_test_lstm, verbose=0).flatten()
        results.append({
            'Symbol': sym,
            'model': 'LSTM',
            'start_idx': start,
            'rmse': np.sqrt(mean_squared_error(y_test, y_pred_lstm)),
            'mae': mean_absolute_error(y_test, y_pred_lstm),
            'r2': r2_score(y_test, y_pred_lstm)
        })

        # ===============================
        # 📈 3. Prophet
        # ===============================
        df_prophet = df_sym.iloc[list(train_idx) + list(test_idx)][['Date', 'Close_next']].rename(columns={'Date': 'ds', 'Close_next': 'y'})
        df_prophet_train = df_prophet.iloc[:len(train_idx)]
        df_prophet_test = df_prophet.iloc[len(train_idx):]

        prophet_model = Prophet(daily_seasonality=True)
        prophet_model.fit(df_prophet_train)

        future = df_prophet_test[['ds']]
        forecast = prophet_model.predict(future)
        y_pred_prophet = forecast['yhat'].values
        y_test_prophet = df_prophet_test['y'].values

        results.append({
            'Symbol': sym,
            'model': 'Prophet',
            'start_idx': start,
            'rmse': np.sqrt(mean_squared_error(y_test_prophet, y_pred_prophet)),
            'mae': mean_absolute_error(y_test_prophet, y_pred_prophet),
            'r2': r2_score(y_test_prophet, y_pred_prophet)
        })

        start += step

# ===============================
# ✅ รวมผลลัพธ์ทั้งหมด
# ===============================
results_df = pd.DataFrame(results)
pd.set_option('display.max_rows', None)
print(results_df)



Feature Matrix X:
   Close_lag1  Close_lag2  Close_lag3  Close_lag4  Close_lag5  Close_lag6  \
0         NaN         NaN         NaN         NaN         NaN         NaN   
1   37.575207         NaN         NaN         NaN         NaN         NaN   
2   33.832436   37.575207         NaN         NaN         NaN         NaN   
3   35.276726   33.832436   37.575207         NaN         NaN         NaN   
4   35.250552   35.276726   33.832436   37.575207         NaN         NaN   
5   35.224378   35.250552   35.276726   33.832436   37.575207         NaN   
6   35.198204   35.224378   35.250552   35.276726   33.832436   37.575207   
7   35.869202   35.198204   35.224378   35.250552   35.276726   33.832436   
8   36.478310   35.869202   35.198204   35.224378   35.250552   35.276726   
9   36.594902   36.478310   35.869202   35.198204   35.224378   35.250552   

   Close_lag7  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN  
5         NaN  
6         NaN  
7   37

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
DEBUG:cmdstanpy:input tempfile: /tmp/tmplu9o6d24/wcv7ru98.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmplu9o6d24/ljhje90y.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=34096', 'data', 'file=/tmp/tmplu9o6d24/wcv7ru98.json', 'init=/tmp/tmplu9o6d24/ljhje90y.json', 'output', 'file=/tmp/tmplu9o6d24/prophet_model2dbe9eo8/prophet_model-20251005070416.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
07:04:16 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:04:17 - cmdstanpy - INFO - Chain [1] done processi

    Symbol             model  start_idx        rmse         mae          r2
0     AAPL            Linear          0    2.518403    1.808699    0.941171
1     AAPL      RandomForest          0   10.808774    8.387513   -0.083670
2     AAPL  GradientBoosting          0   11.100681    8.592792   -0.142992
3     AAPL           XGBoost          0    9.564359    7.456274    0.151492
4     AAPL               SVR          0   22.012600   18.252869   -3.494555
5     AAPL              LSTM          0   42.315998   38.932482  -15.609369
6     AAPL           Prophet          0   58.504814   56.748644  -30.748790
7     AAPL            Linear        327    3.339538    2.374465    0.983370
8     AAPL      RandomForest        327   27.130128   20.951063   -0.097564
9     AAPL  GradientBoosting        327   27.402882   21.120182   -0.119744
10    AAPL           XGBoost        327   27.853443   21.678828   -0.156868
11    AAPL               SVR        327   59.797778   45.897897   -4.332074
12    AAPL  

In [ ]:
# Export to Excel
results_df.to_excel("Model  4 Linear_interp_lag7_5 models.xlsx", index=False)
print("✅ Exported results to Model  4 Linear_interp_lag7_5 models.xlsx")

✅ Exported results to Model  4 Linear_interp_lag7_5 models.xlsx
